# FlightRank 2025 - Baseline Model
Previsão de seleção de voos com LightGBM (Lambdarank)

---

## 1. Imports e Configuração Inicial

Importação de bibliotecas e definição de opções globais.

In [1]:
import pandas as pd
import os
import subprocess
import zipfile
import matplotlib.pyplot as plt
import lightgbm as lgb
import numpy as np
import lightgbm as lgb

# --- Split com GroupShuffleSplit
from sklearn.model_selection import GroupShuffleSplit
from sklearn.model_selection import GroupShuffleSplit
from itertools import chain
from sklearn.model_selection import GroupKFold
# --- LightGBM Dataset
import lightgbm as lgb

In [2]:
# resetando as configurações 
pd.reset_option('display.max_columns')

# setando as configurações de coluna maxima
# importante pois tem muitas colunas
pd.set_option('display.max_columns', None)

## 2. Download e extração dos dados
Verifica se os arquivos da competição já existem localmente, caso contrário, baixa e extrai os dados.


In [3]:
def download_files():
    # utilizando a API do kaggle para download
    # adicionado a /data no git ignore
    # Define caminhos
    zip_path = "data/aeroclub-recsys-2025.zip"
    extract_path = "data/aeroclub"
    
    # Cria a pasta base se necessário
    os.makedirs("data", exist_ok=True)

    # Verifica se o arquivo .zip já foi baixado
    if not os.path.exists(zip_path):
        print("🔽 Baixando arquivos da competição...")
        subprocess.run([
            "kaggle", "competitions", "download",
            "-c", "aeroclub-recsys-2025",
            "-p", "data"
        ])
    else:
        print("✅ Arquivo ZIP já existe. Pulando download.")

    # Verifica se os arquivos já foram extraídos
    if not os.path.exists(extract_path) or not os.listdir(extract_path):
        print("📦 Extraindo arquivos...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
    else:
        print("✅ Arquivos já extraídos. Pulando extração.")

In [4]:
# Executa
download_files()

✅ Arquivo ZIP já existe. Pulando download.
✅ Arquivos já extraídos. Pulando extração.


## 3. Leitura dos dados
Lê o arquivo `train.parquet` com o Pandas.


In [5]:
train = pd.read_parquet("data/aeroclub/train.parquet")

In [6]:
def reduce_memory_usage(df):
    for col in df.columns:
        col_type = df[col].dtypes
        
        if col_type == 'float64':
            df[col] = pd.to_numeric(df[col], downcast='float')
        elif col_type == 'int64':
            df[col] = pd.to_numeric(df[col], downcast='integer')
        elif col_type == 'object':
            num_unique = df[col].nunique()
            num_total = len(df[col])
            if num_unique / num_total < 0.5:
                df[col] = df[col].astype('category')
    
    return df
train = reduce_memory_usage(train)

In [7]:
df_train_raw = train.copy()

## 4. Seleção de colunas relevantes
Seleciona apenas as colunas que serão usadas no baseline.


In [8]:
# Define as colunas que você quer manter
columns_to_keep = [
    # Identifiers
    'Id',  # num
    'ranker_id', 
    'profileId', 
    'companyID',
    
    # User info
    'sex', 'nationality', 'frequentFlyer', 'isVip', 'bySelf', 'isAccess3D',

    # Company info
    'corporateTariffCode',

    # Search & route
    'searchRoute', 'requestDate',

    # Pricing
    'totalPrice', 'taxes',

    # Flight timing
    'legs0_departureAt', 'legs0_arrivalAt', 'legs0_duration',
    'legs1_departureAt', 'legs1_arrivalAt', 'legs1_duration',

    # Segment-level info (só do segmento 0 da ida para simplificar no baseline)
    'legs0_segments0_departureFrom_airport_iata',
    'legs0_segments0_arrivalTo_airport_iata',
    'legs0_segments0_arrivalTo_airport_city_iata',
    'legs0_segments0_marketingCarrier_code',
    'legs0_segments0_operatingCarrier_code',
    'legs0_segments0_aircraft_code',
    'legs0_segments0_flightNumber',
    'legs0_segments0_duration',
    'legs0_segments0_baggageAllowance_quantity',
    'legs0_segments0_baggageAllowance_weightMeasurementType',
    'legs0_segments0_cabinClass',
    'legs0_segments0_seatsAvailable',

    # Cancellation & exchange rules
    'miniRules0_monetaryAmount', 'miniRules0_percentage', 'miniRules0_statusInfos',
    'miniRules1_monetaryAmount', 'miniRules1_percentage', 'miniRules1_statusInfos',

    # Pricing policy
    'pricingInfo_isAccessTP', 'pricingInfo_passengerCount',

    # Target
    'selected'
]

# Filtra os dados para o baseline
rows_to_copy = 1_000_000
rows_to_copy = len(df_train_raw)
print(f"rows to read: {rows_to_copy}")
df_train = df_train_raw[columns_to_keep].iloc[:rows_to_copy].copy()

rows to read: 18145372


### 5. Engenharia de features (corrige dtypes)
Corrige os dtypes

In [9]:
def fix_column_types(df):
    df_fixed = df.copy()
    for col in df.columns:
        if isinstance(df[col].dtype, pd.CategoricalDtype):
            # Tenta converter para tipo numérico
            try:
                df_fixed[col] = pd.to_numeric(df[col])
            except:
                # Se não for numérico, tenta bool
                unique_vals = df[col].dropna().unique()
                if set(unique_vals) <= {True, False}:
                    df_fixed[col] = df[col].astype(bool)
                else:
                    df_fixed[col] = df[col].astype(str)
    return df_fixed
df_train = fix_column_types(df_train)

# Ajusta a nacionalidade (está em Int)
df_train["nationality"] = df_train["nationality"].astype("str")


df_train.dtypes  # Checar resultado

Id                                                                 int32
ranker_id                                                         object
profileId                                                          int32
companyID                                                          int32
sex                                                                 bool
nationality                                                       object
frequentFlyer                                                     object
isVip                                                               bool
bySelf                                                              bool
isAccess3D                                                          bool
corporateTariffCode                                                Int64
searchRoute                                                       object
requestDate                                               datetime64[ns]
totalPrice                                         

## 5. Engenharia de features
Divisão da coluna FrenquentFlyer


In [10]:
def count_frequent_flyers(value):
    if pd.isna(value):
        return 0
    return len(str(value).split('/'))

df_train['frequentFlyer_count'] = df_train['frequentFlyer'].apply(count_frequent_flyers)

# Cria flag binária para frequent flyer
df_train['hasFrequentFlyer'] = df_train['frequentFlyer'].notnull().astype(int)

# Substituir valores NaN por string vazia
ff_series = df_train['frequentFlyer'].fillna('').astype(str)

# Dividir por '/' para obter lista
ff_lists = ff_series.str.split('/')

all_programs = set(chain.from_iterable(ff_lists))
print(f"Total de companhias únicas: {len(all_programs)}")


Total de companhias únicas: 71


In [11]:
"""
for program in all_programs:
    if program == '':
        continue  # pula string vazia
    df_train[f'ff_{program}'] = ff_lists.apply(lambda x: int(program in x))

for col in df_train.columns:
    if col.startswith("ff_"):
        df_train[col] = df_train[col].astype(pd.BooleanDtype())
"""

'\nfor program in all_programs:\n    if program == \'\':\n        continue  # pula string vazia\n    df_train[f\'ff_{program}\'] = ff_lists.apply(lambda x: int(program in x))\n\nfor col in df_train.columns:\n    if col.startswith("ff_"):\n        df_train[col] = df_train[col].astype(pd.BooleanDtype())\n'

In [12]:
df_train['searchRoute'].head()

0    TLKKJA/KJATLK
1    TLKKJA/KJATLK
2    TLKKJA/KJATLK
3    TLKKJA/KJATLK
4    TLKKJA/KJATLK
Name: searchRoute, dtype: object

In [13]:
df_train.drop('frequentFlyer', axis=1, inplace=True)

## 5. Engenharia de features
Timedelta Columns


In [14]:
#df_train['legs0_departureAt']

In [15]:
# 🗓️ Colunas de datas e horários
cols_datetime = [
    'requestDate',
    'legs0_departureAt', 'legs0_arrivalAt',
    'legs1_departureAt', 'legs1_arrivalAt'
]
def process_datetime_and_duration(df):
    df_processed = df.copy()

    # Datas para datetime
    for col in cols_datetime:
        df_processed[col] = pd.to_datetime(df_processed[col], errors='coerce')

    # Features de hora e dia da semana
    df_processed['legs0_dep_hour'] = df_processed['legs0_departureAt'].dt.hour
    df_processed['legs0_dep_dayofweek'] = df_processed['legs0_departureAt'].dt.dayofweek
    df_processed['legs1_dep_hour'] = df_processed['legs1_departureAt'].dt.hour
    df_processed['legs1_dep_dayofweek'] = df_processed['legs1_departureAt'].dt.dayofweek

    # Dias entre ida e volta (duração da viagem)
    df_processed['trip_days'] = (df_processed['legs1_departureAt'] - df_processed['legs0_departureAt']).dt.days

    # Dias de antecedência (request → ida)
    df_processed['booking_to_trip_days'] = (df_processed['legs0_departureAt'] - df_processed['requestDate']).dt.days

    # Final de semana (ida/volta)
    df_processed['ida_fds'] = df_processed['legs0_dep_dayofweek'].isin([5, 6]).astype(int)
    df_processed['volta_fds'] = df_processed['legs1_dep_dayofweek'].isin([5, 6]).astype(int)

    # Horário comercial (7h às 19h)
    def is_business_hour(hour):
        return int(7 <= hour <= 19)

    df_processed['ida_comercial'] = df_processed['legs0_dep_hour'].apply(is_business_hour)
    df_processed['volta_comercial'] = df_processed['legs1_dep_hour'].apply(is_business_hour)

    # ⏱️ Converter colunas de duração para minutos
    def clean_and_convert_duration(col):
        return (
            col
            .fillna("00:00:00")
            .astype(str)
            .str.strip()
            .str.replace("nan", "00:00:00")
            .pipe(pd.to_timedelta, errors='coerce')
            .dt.total_seconds() / 60  # minutos
        )

    cols_duration = ['legs0_duration', 'legs1_duration']
    for col in cols_duration:
        df_processed[col] = clean_and_convert_duration(df_processed[col])

    return df_processed



In [16]:
df_train['legs0_duration_minutes'] = (
    pd.to_timedelta(
        df_train['legs0_segments0_duration'].fillna("00:00:00").astype(str).str.strip(),
        errors='coerce'
    ).dt.total_seconds() / 60  # em minutos
)

df_train.drop('legs0_segments0_duration', axis=1, inplace=True)

In [17]:
df_train['legs0_duration_minutes']

0           160.0
1           170.0
2           170.0
3           170.0
4           170.0
            ...  
18146427    200.0
18146428    125.0
18146429    125.0
18146430    105.0
18146431    105.0
Name: legs0_duration_minutes, Length: 18145372, dtype: float64

In [18]:
df_train

,Id,ranker_id,profileId,companyID,sex,nationality,isVip,bySelf,isAccess3D,corporateTariffCode,searchRoute,requestDate,totalPrice,taxes,legs0_departureAt,legs0_arrivalAt,legs0_duration,legs1_departureAt,legs1_arrivalAt,legs1_duration,legs0_segments0_departureFrom_airport_iata,legs0_segments0_arrivalTo_airport_iata,legs0_segments0_arrivalTo_airport_city_iata,legs0_segments0_marketingCarrier_code,legs0_segments0_operatingCarrier_code,legs0_segments0_aircraft_code,legs0_segments0_flightNumber,legs0_segments0_baggageAllowance_quantity,legs0_segments0_baggageAllowance_weightMeasurementType,legs0_segments0_cabinClass,legs0_segments0_seatsAvailable,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,selected,frequentFlyer_count,hasFrequentFlyer,legs0_duration_minutes
0,0,98ce0dabf6964640b63079fbafd42cbe,2087645,57323,True,36,False,True,False,<NA>,TLKKJA/KJATLK,2024-05-17 03:03:08,16884.0,370.0,2024-06-15T15:40:00,2024-06-15T16:20:00,02:40:00,2024-07-09T09:45:00,2024-07-09T14:20:00,02:35:00,TLK,KJA,KJA,KV,KV,YK2,216,1.0,0.0,1.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1,1,3,1,160.0
1,1,98ce0dabf6964640b63079fbafd42cbe,2087645,57323,True,36,False,True,True,123,TLKKJA/KJATLK,2024-05-17 03:03:08,51125.0,2240.0,2024-06-15T09:25:00,2024-06-15T14:50:00,07:25:00,2024-07-09T22:05:00,2024-07-10T08:30:00,08:25:00,TLK,OVB,OVB,S7,S7,E70,5358,1.0,0.0,1.0,4.0,2300.0,NaN,1.0,3500.0,NaN,1.0,1.0,1,0,3,1,170.0
2,2,98ce0dabf6964640b63079fbafd42cbe,2087645,57323,True,36,False,True,False,<NA>,TLKKJA/KJATLK,2024-05-17 03:03:08,53695.0,2240.0,2024-06-15T09:25:00,2024-06-15T14:50:00,07:25:00,2024-07-09T22:05:00,2024-07-10T08:30:00,08:25:00,TLK,OVB,OVB,S7,S7,E70,5358,1.0,0.0,1.0,4.0,2300.0,NaN,1.0,3500.0,NaN,1.0,1.0,1,0,3,1,170.0
3,3,98ce0dabf6964640b63079fbafd42cbe,2087645,57323,True,36,False,True,True,123,TLKKJA/KJATLK,2024-05-17 03:03:08,81880.0,2240.0,2024-06-15T09:25:00,2024-06-15T14:50:00,07:25:00,2024-07-09T22:05:00,2024-07-10T08:30:00,08:25:00,TLK,OVB,OVB,S7,S7,E70,5358,1.0,0.0,1.0,4.0,0.0,NaN,1.0,0.0,NaN,1.0,1.0,1,0,3,1,170.0
4,4,98ce0dabf6964640b63079fbafd42cbe,2087645,57323,True,36,False,True,False,<NA>,TLKKJA/KJATLK,2024-05-17 03:03:08,86070.0,2240.0,2024-06-15T09:25:00,2024-06-15T14:50:00,07:25:00,2024-07-09T22:05:00,2024-07-10T08:30:00,08:25:00,TLK,OVB,OVB,S7,S7,E70,5358,1.0,0.0,1.0,4.0,0.0,NaN,1.0,0.0,NaN,1.0,1.0,1,0,3,1,170.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18146427,18146427,88f8c53a28bf4f438941fd67338009e8,3046852,54154,True,36,False,True,True,44,MOWSVX/SVXMOW,2024-10-29 12:46:20,30730.0,5560.0,2024-11-05T20:50:00,2024-11-06T09:45:00,10:55:00,2024-11-09T19:25:00,2024-11-10T08:10:00,14:45:00,VKO,SGC,SGC,UT,UT,738,247,0.0,0.0,1.0,3.0,24000.0,NaN,1.0,0.0,NaN,0.0,0.0,1,0,1,1,200.0
18146428,18146428,88f8c53a28bf4f438941fd67338009e8,3046852,54154,True,36,False,True,True,44,MOWSVX/SVXMOW,2024-10-29 12:46:20,27660.0,5560.0,2024-11-05T00:20:00,2024-11-05T20:00:00,17:40:00,2024-11-09T21:10:00,2024-11-10T08:35:00,13:25:00,VKO,UFA,UFA,UT,UT,738,363,0.0,0.0,1.0,3.0,24000.0,NaN,1.0,0.0,NaN,0.0,0.0,1,0,1,1,125.0
18146429,18146429,88f8c53a28bf4f438941fd67338009e8,3046852,54154,True,36,False,True,True,44,MOWSVX/SVXMOW,2024-10-29 12:46:20,24460.0,5460.0,2024-11-05T00:20:00,2024-11-05T20:00:00,17:40:00,2024-11-09T19:25:00,2024-11-10T08:10:00,14:45:00,VKO,UFA,UFA,UT,UT,738,363,0.0,0.0,1.0,3.0,24000.0,NaN,1.0,0.0,NaN,0.0,0.0,1,0,1,1,125.0
18146430,18146430,88f8c53a28bf4f438941fd67338009e8,3046852,54154,True,36,False,True,True,44,MOWSVX/SVXMOW,2024-10-29 12:46:20,25360.0,5560.0,2024-11-05T19:30:00,2024-11-06T17:10:00,19:40:00,2024-11-09T21:10:00,2024-11-10T08:35:00,13:25:00,VKO,KUF,KUF,UT,UT,738,357,0.0,0.0,1.0,4.0,24000.0,NaN,1.0,0.0,NaN,0.0,0.0,1,0,1,1,105.0


In [19]:
# ✅ Applicação
df_train = process_datetime_and_duration(df_train)

In [20]:
#df_train.drop(columns=cols_datetime, inplace=True)

## 5. Engenharia de features
booleans


In [21]:
bool_cols = [
    'pricingInfo_isAccessTP',
    'hasFrequentFlyer',
]

for col in bool_cols:
    df_train[col] = df_train[col].astype('boolean')

In [22]:
df_train

,Id,ranker_id,profileId,companyID,sex,nationality,isVip,bySelf,isAccess3D,corporateTariffCode,searchRoute,requestDate,totalPrice,taxes,legs0_departureAt,legs0_arrivalAt,legs0_duration,legs1_departureAt,legs1_arrivalAt,legs1_duration,legs0_segments0_departureFrom_airport_iata,legs0_segments0_arrivalTo_airport_iata,legs0_segments0_arrivalTo_airport_city_iata,legs0_segments0_marketingCarrier_code,legs0_segments0_operatingCarrier_code,legs0_segments0_aircraft_code,legs0_segments0_flightNumber,legs0_segments0_baggageAllowance_quantity,legs0_segments0_baggageAllowance_weightMeasurementType,legs0_segments0_cabinClass,legs0_segments0_seatsAvailable,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,selected,frequentFlyer_count,hasFrequentFlyer,legs0_duration_minutes,legs0_dep_hour,legs0_dep_dayofweek,legs1_dep_hour,legs1_dep_dayofweek,trip_days,booking_to_trip_days,ida_fds,volta_fds,ida_comercial,volta_comercial
0,0,98ce0dabf6964640b63079fbafd42cbe,2087645,57323,True,36,False,True,False,<NA>,TLKKJA/KJATLK,2024-05-17 03:03:08,16884.0,370.0,2024-06-15 15:40:00,2024-06-15 16:20:00,160.0,2024-07-09 09:45:00,2024-07-09 14:20:00,155.0,TLK,KJA,KJA,KV,KV,YK2,216,1.0,0.0,1.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,True,1,1,3,True,160.0,15,5,9.0,1.0,23.0,29,1,0,1,1
1,1,98ce0dabf6964640b63079fbafd42cbe,2087645,57323,True,36,False,True,True,123,TLKKJA/KJATLK,2024-05-17 03:03:08,51125.0,2240.0,2024-06-15 09:25:00,2024-06-15 14:50:00,445.0,2024-07-09 22:05:00,2024-07-10 08:30:00,505.0,TLK,OVB,OVB,S7,S7,E70,5358,1.0,0.0,1.0,4.0,2300.0,NaN,1.0,3500.0,NaN,1.0,True,1,0,3,True,170.0,9,5,22.0,1.0,24.0,29,1,0,1,0
2,2,98ce0dabf6964640b63079fbafd42cbe,2087645,57323,True,36,False,True,False,<NA>,TLKKJA/KJATLK,2024-05-17 03:03:08,53695.0,2240.0,2024-06-15 09:25:00,2024-06-15 14:50:00,445.0,2024-07-09 22:05:00,2024-07-10 08:30:00,505.0,TLK,OVB,OVB,S7,S7,E70,5358,1.0,0.0,1.0,4.0,2300.0,NaN,1.0,3500.0,NaN,1.0,True,1,0,3,True,170.0,9,5,22.0,1.0,24.0,29,1,0,1,0
3,3,98ce0dabf6964640b63079fbafd42cbe,2087645,57323,True,36,False,True,True,123,TLKKJA/KJATLK,2024-05-17 03:03:08,81880.0,2240.0,2024-06-15 09:25:00,2024-06-15 14:50:00,445.0,2024-07-09 22:05:00,2024-07-10 08:30:00,505.0,TLK,OVB,OVB,S7,S7,E70,5358,1.0,0.0,1.0,4.0,0.0,NaN,1.0,0.0,NaN,1.0,True,1,0,3,True,170.0,9,5,22.0,1.0,24.0,29,1,0,1,0
4,4,98ce0dabf6964640b63079fbafd42cbe,2087645,57323,True,36,False,True,False,<NA>,TLKKJA/KJATLK,2024-05-17 03:03:08,86070.0,2240.0,2024-06-15 09:25:00,2024-06-15 14:50:00,445.0,2024-07-09 22:05:00,2024-07-10 08:30:00,505.0,TLK,OVB,OVB,S7,S7,E70,5358,1.0,0.0,1.0,4.0,0.0,NaN,1.0,0.0,NaN,1.0,True,1,0,3,True,170.0,9,5,22.0,1.0,24.0,29,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18146427,18146427,88f8c53a28bf4f438941fd67338009e8,3046852,54154,True,36,False,True,True,44,MOWSVX/SVXMOW,2024-10-29 12:46:20,30730.0,5560.0,2024-11-05 20:50:00,2024-11-06 09:45:00,655.0,2024-11-09 19:25:00,2024-11-10 08:10:00,885.0,VKO,SGC,SGC,UT,UT,738,247,0.0,0.0,1.0,3.0,24000.0,NaN,1.0,0.0,NaN,0.0,False,1,0,1,True,200.0,20,1,19.0,5.0,3.0,7,0,1,0,1
18146428,18146428,88f8c53a28bf4f438941fd67338009e8,3046852,54154,True,36,False,True,True,44,MOWSVX/SVXMOW,2024-10-29 12:46:20,27660.0,5560.0,2024-11-05 00:20:00,2024-11-05 20:00:00,1060.0,2024-11-09 21:10:00,2024-11-10 08:35:00,805.0,VKO,UFA,UFA,UT,UT,738,363,0.0,0.0,1.0,3.0,24000.0,NaN,1.0,0.0,NaN,0.0,False,1,0,1,True,125.0,0,1,21.0,5.0,4.0,6,0,1,0,0
18146429,18146429,88f8c53a28bf4f438941fd67338009e8,3046852,54154,True,36,False,True,True,44,MOWSVX/SVXMOW,2024-10-29 12:46:20,24460.0,5460.0,2024-11-05 00:20:00,2024-11-05 20:00:00,1060.0,2024-11-09 19:25:00,2024-11-10 08:10:00,885.0,VKO,UFA,UFA,UT,UT,738,363,0.0,0.0,1.0,3.0,24000.0,NaN,1.0,0.0,NaN,0.0,False,1

SEACH ROUTE

In [23]:
df_train['searchRoute'] = df_train['searchRoute'].astype(str)
df_train['searchRoute_count'] = df_train['searchRoute'].apply(lambda x: x.split("/"))
df_train['searchRoute_count'] = df_train['searchRoute_count'].apply(lambda x: len(x))
print(f" min {min(df_train['searchRoute_count'])}")
print(f" max {max(df_train['searchRoute_count'])}")
df_train.drop('searchRoute_count', axis=1, inplace=True)

 min 1
 max 2


In [24]:
# Garante que searchRoute está como string
df_train['searchRoute'] = df_train['searchRoute'].astype(str)

# Separa ida e volta
df_train[['route_ida', 'route_volta']] = df_train['searchRoute'].str.split('/', expand=True)

# Extrai origem e destino da ida
df_train['ida_from'] = df_train['route_ida'].str[:3]
df_train['ida_to'] = df_train['route_ida'].str[3:]

# Extrai origem e destino da volta (se existir)
df_train['volta_from'] = df_train['route_volta'].str[:3]
df_train['volta_to'] = df_train['route_volta'].str[3:]

df_train.drop('searchRoute', axis=1, inplace=True)

In [25]:
# Ver todos os dtypes
with pd.option_context('display.max_rows', None):
    display(df_train.dtypes)

Id                                                                 int32
ranker_id                                                         object
profileId                                                          int32
companyID                                                          int32
sex                                                                 bool
nationality                                                       object
isVip                                                               bool
bySelf                                                              bool
isAccess3D                                                          bool
corporateTariffCode                                                Int64
requestDate                                               datetime64[ns]
totalPrice                                                       float32
taxes                                                            float32
legs0_departureAt                                  

In [26]:
# --- Target e grupo
target_col = "selected"
group_col = "ranker_id"

# --- Categóricas para LightGBM
categorical_cols = [
    'nationality',
    'legs0_segments0_departureFrom_airport_iata',
    'legs0_segments0_arrivalTo_airport_iata',
    'legs0_segments0_arrivalTo_airport_city_iata',
    'legs0_segments0_marketingCarrier_code',
    'legs0_segments0_operatingCarrier_code',
    'legs0_segments0_aircraft_code',
    'corporateTariffCode',
    
    # novas features categóricas da searchRoute
    'route_ida',
    'route_volta',
    'ida_from',
    'ida_to',
    'volta_from',
    'volta_to'
]

# --- Booleanas e numéricas
boolean_cols = [
    'sex', 'isVip', 'bySelf', 'isAccess3D',
    'pricingInfo_isAccessTP', 'hasFrequentFlyer',
    'ida_fds', 'volta_fds',
    'ida_comercial', 'volta_comercial'
] + [col for col in df_train.columns if col.startswith("ff_")]

numeric_cols = [
    'totalPrice', 'taxes',
    'legs0_duration', 'legs1_duration',
    #'legs0_segments0_duration',
    'legs0_segments0_baggageAllowance_quantity',
    'legs0_segments0_baggageAllowance_weightMeasurementType',
    'legs0_segments0_cabinClass',
    'legs0_segments0_seatsAvailable',
    'miniRules0_monetaryAmount', 'miniRules0_percentage',
    'miniRules1_monetaryAmount', 'miniRules1_percentage',
    'booking_to_trip_days', 'trip_days',
    'legs0_dep_hour', 'legs0_dep_dayofweek',
    'legs1_dep_hour', 'legs1_dep_dayofweek',
    'frequentFlyer_count', 'legs0_duration_minutes'
]
features = numeric_cols + categorical_cols + boolean_cols

# --- Converte categóricas para category
for col in categorical_cols:
    df_train[col] = df_train[col].astype("category")





In [27]:
# --- Separação por grupo (ranker_id)
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(gss.split(df_train, groups=df_train["ranker_id"]))

df_train_split = df_train.iloc[train_idx].copy()
df_val = df_train.iloc[val_idx].copy()

# --- Features e targets
X_train = df_train_split[features]
y_train = df_train_split[target_col]
groups_train = df_train_split[group_col].value_counts().sort_index().values

X_val = df_val[features]
y_val = df_val[target_col]
groups_val = df_val[group_col].value_counts().sort_index().values

# --- Parâmetros que afetam o Dataset (incluindo GPU e max_bin!)
dataset_params = {
    "max_bin": 63,  # ou 31, se continuar com erro
    "device": "gpu"
}

# --- Criação dos Datasets
train_dataset = lgb.Dataset(
    X_train,
    label=y_train,
    group=groups_train,
    categorical_feature=categorical_cols,
    params=dataset_params  # 💡 AQUI é onde max_bin deve ir também!
)

val_dataset = lgb.Dataset(
    X_val,
    label=y_val,
    group=groups_val,
    categorical_feature=categorical_cols,
    reference=train_dataset,
    params=dataset_params
)



In [29]:
# --- Parâmetros
params = {
    "objective": "lambdarank",
    "metric": "ndcg",
    "ndcg_eval_at": [3],
    "learning_rate": 0.05,
    "num_leaves": 31,
    "min_data_in_leaf": 20,
    "verbosity": -1,
}
"""
params = {
    "objective": "lambdarank",
    "metric": "ndcg",
    "ndcg_eval_at": [1, 3, 5, 10],
    "device": "gpu",
    "gpu_platform_id": 0,
    "gpu_device_id": 0,
    "force_col_wise": True,
    "max_bin": 255,  # 🔧 necessário para rodar na GPU
    "learning_rate": 0.05,
    "num_leaves": 128,
    "max_depth": -1,
    "verbosity": -1
}
"""

# --- Treinamento com early stopping
model = lgb.train(
    params,
    train_dataset,
    valid_sets=[train_dataset, val_dataset],
    valid_names=["train", "valid"],
    num_boost_round=1000,
    callbacks=[lgb.early_stopping(stopping_rounds=50)],
    #verbose_eval=50
)

# --- Predição
y_pred = model.predict(X_val)

# --- Avaliação Top-1
df_pred = df_val.copy()
df_pred['y_true'] = y_val
df_pred['y_pred'] = y_pred

df_pred_sorted = df_pred.sort_values(['ranker_id', 'y_pred'], ascending=[True, False])
df_top1 = df_pred_sorted.groupby('ranker_id').head(1)

acertos = df_top1['y_true'].sum()
total = df_top1.shape[0]

print(f"Voos escolhidos corretamente (top1): {acertos} de {total} sessões")
print(f"Acurácia top1: {acertos / total:.4f}")


C:\Users\lpl_z\anaconda3\envs\flightrec-env\lib\site-packages\lightgbm\basic.py:2535: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning("Overriding the parameters from Reference Dataset.")


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[699]	train's ndcg@3: 0.862131	valid's ndcg@3: 0.828594
Voos escolhidos corretamente (top1): 7045 de 21108 sessões
Acurácia top1: 0.3338


In [30]:
# ============================================================
# ✅ Treinamento final com TODO o dataset de treino
#     usando best_iteration encontrado na validação
# ============================================================

X_full = df_train[features]
y_full = df_train[target_col]
groups_full = df_train[group_col].value_counts().sort_index().values

full_dataset = lgb.Dataset(X_full, y_full, group=groups_full, categorical_feature=categorical_cols)

# ⚠️ Usa o número ideal de iterações do treino anterior
final_model = lgb.train(
    params,
    full_dataset,
    num_boost_round=model.best_iteration  # << Aqui está a mágica
)


In [ ]:
# ============================================================
# ## 6. Geração de Submissão
# ============================================================

# 1. Ler test.parquet
df_test = pd.read_parquet("data/aeroclub/test.parquet")

# 2. Aplicar transformações mínimas necessárias
df_test['ranker_id'] = df_test['ranker_id'].astype(str)
df_test['nationality'] = df_test['nationality'].astype(str)
df_test['searchRoute'] = df_test['searchRoute'].astype(str)

# --- Frequent Flyer (mesmos one-hot do treino)
df_test['frequentFlyer'] = df_test['frequentFlyer'].fillna('').astype(str)
ff_lists_test = df_test['frequentFlyer'].str.split('/')

for program in all_programs:
    if program == '':
        continue
    df_test[f'ff_{program}'] = ff_lists_test.apply(lambda x: int(program in x))

for col in [col for col in df_test.columns if col.startswith("ff_")]:
    df_test[col] = df_test[col].astype(pd.BooleanDtype())

df_test['frequentFlyer_count'] = df_test['frequentFlyer'].apply(count_frequent_flyers)
df_test['hasFrequentFlyer'] = df_test['frequentFlyer'].notnull().astype(int)
df_test.drop(columns=['frequentFlyer'], inplace=True)

# --- Datas
cols_datetime = [
    'requestDate',
    'legs0_departureAt', 'legs0_arrivalAt',
    'legs1_departureAt', 'legs1_arrivalAt'
]
for col in cols_datetime:
    df_test[col] = pd.to_datetime(df_test[col], errors='coerce')

df_test['legs0_dep_hour'] = df_test['legs0_departureAt'].dt.hour
df_test['legs0_dep_dayofweek'] = df_test['legs0_departureAt'].dt.dayofweek
df_test['legs1_dep_hour'] = df_test['legs1_departureAt'].dt.hour
df_test['legs1_dep_dayofweek'] = df_test['legs1_departureAt'].dt.dayofweek
df_test['trip_days'] = (df_test['legs1_departureAt'] - df_test['legs0_departureAt']).dt.days
df_test['booking_to_trip_days'] = (df_test['legs0_departureAt'] - df_test['requestDate']).dt.days
df_test['ida_fds'] = df_test['legs0_dep_dayofweek'].isin([5, 6]).astype(int)
df_test['volta_fds'] = df_test['legs1_dep_dayofweek'].isin([5, 6]).astype(int)

df_test['ida_comercial'] = df_test['legs0_dep_hour'].apply(lambda x: int(7 <= x <= 19))
df_test['volta_comercial'] = df_test['legs1_dep_hour'].apply(lambda x: int(7 <= x <= 19))

df_test.drop(columns=cols_datetime, inplace=True)

# --- Duração
def clean_and_convert_duration(col):
    return (
        col
        .fillna("00:00:00")
        .astype(str)
        .str.strip()
        .str.replace("nan", "00:00:00")
        .pipe(pd.to_timedelta, errors='coerce')
        .dt.total_seconds() / 60
    )

df_test['legs0_duration'] = clean_and_convert_duration(df_test['legs0_duration'])
df_test['legs1_duration'] = clean_and_convert_duration(df_test['legs1_duration'])
df_test['legs0_segments0_duration'] = clean_and_convert_duration(df_test['legs0_segments0_duration'])
df_test['legs0_duration_minutes'] = df_test['legs0_duration']
df_test.drop(columns=['legs0_segments0_duration'], inplace=True)

# --- SearchRoute features
df_test[['route_ida', 'route_volta']] = df_test['searchRoute'].str.split('/', expand=True)
df_test['ida_from'] = df_test['route_ida'].str[:3]
df_test['ida_to'] = df_test['route_ida'].str[3:]
df_test['volta_from'] = df_test['route_volta'].str[:3]
df_test['volta_to'] = df_test['route_volta'].str[3:]
df_test.drop('searchRoute', axis=1, inplace=True)

# --- Tipagem
for col in categorical_cols:
    df_test[col] = df_test[col].astype("category")

for col in boolean_cols:
    if col in df_test.columns:
        df_test[col] = df_test[col].astype('boolean')

# 3. Prever com o modelo
X_test = df_test[features]
df_test['y_pred'] = model.predict(X_test)

# 4. Gerar submissão
df_test_sorted = df_test.sort_values(['ranker_id', 'y_pred'], ascending=[True, False])
df_test_sorted['selected'] = df_test_sorted.groupby('ranker_id').cumcount() + 1

submission = df_test_sorted[['Id', 'ranker_id', 'selected']]
submission.to_csv("submission.csv", index=False)
print("✅ Arquivo de submissão salvo como 'submission.csv'")
